# Visualization benchmark data

In [ ]:
# Imports
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np

In [ ]:
import matplotlib.font_manager as fm

font_path = "./fonts/UGentPannoText-Normal.ttf"
fm.fontManager.addfont(font_path)
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()

font_path = "./fonts/UGentPannoText-Medium.ttf"
fm.fontManager.addfont(font_path)
medium_font_prop = fm.FontProperties(fname=font_path)

In [ ]:
# Requesting data
from parsing import parse_results_file


def extract_results(file_path: str):
    cases = parse_results_file(file_path)

    # Extract values
    get_latest = [case.duration_get_latest for case in cases]
    direct = [case.duration_direct for case in cases]

    mean_get_latest = np.mean(get_latest, axis=1)
    std_get_latest = np.std(get_latest, axis=1)
    mean_direct = np.mean(direct, axis=1)
    std_direct = np.std(direct, axis=1)

    return cases, mean_get_latest, std_get_latest, mean_direct, std_direct

In [ ]:
def bar_plot(
    x: list[int],
    mean_get_latest: np.ndarray[float],
    std_get_latest: np.ndarray[float],
    mean_direct: np.ndarray[float],
    std_direct: np.ndarray[float],
    x_label: str,
    output_file: str,
    show: bool = True,
):
    width = 0.35  # width of the bars
    x_pos = np.arange(len(x))  # bar positions

    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot bars for get_latest and direct durations
    ax.bar(
        x_pos - width / 2,
        mean_get_latest,
        width,
        yerr=std_get_latest,
        capsize=5,
        label="get_latest_with_secondaries",
        color="#ffe9a2ff",
        alpha=1,
    )
    ax.bar(
        x_pos + width / 2,
        mean_direct,
        width,
        yerr=std_direct,
        capsize=5,
        label="Direct LIST requests",
        color="#689CC7FF",
        alpha=1,
    )

    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f"{y / 1000:.0f}"))

    # Labeling
    ax.set_xlabel(x_label, fontproperties=medium_font_prop, fontsize=18)
    ax.set_ylabel("Duration (ms)", fontproperties=medium_font_prop, fontsize=18)
    ax.set_xticks(x_pos)
    ax.tick_params(axis="x", labelsize=15)
    ax.tick_params(axis="y", labelsize=15)
    ax.set_xticklabels(x)
    ax.legend(fontsize=16, loc="upper right")
    ax.grid(True, axis="y", linestyle="--", alpha=0.6)

    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches="tight")
    if show:
        plt.show()
    else:
        plt.close(fig)

In [ ]:
def compare_plot(
    x: list[int],
    mean_antiaffinity_get_latest: np.ndarray[float],
    std_antiaffinity_get_latest: np.ndarray[float],
    mean_affinity_get_latest: np.ndarray[float],
    std_affinity_get_latest: np.ndarray[float],
    mean_internal_get_latest: np.ndarray[float],
    std_internal_get_latest: np.ndarray[float],
    mean_external_get_latest: np.ndarray[float],
    std_external_get_latest: np.ndarray[float],
    x_label: str,
    output_file: str,
):
    x_pos = np.arange(len(x))  # bar positions
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(mean_antiaffinity_get_latest, label="GKE: different node")
    ax.fill_between(
        x_pos,
        mean_antiaffinity_get_latest - std_antiaffinity_get_latest,
        mean_antiaffinity_get_latest + std_antiaffinity_get_latest,
        alpha=0.15,
    )
    ax.plot(mean_affinity_get_latest, label="GKE: same node")
    ax.fill_between(
        x_pos,
        mean_affinity_get_latest - std_affinity_get_latest,
        mean_affinity_get_latest + std_affinity_get_latest,
        alpha=0.15,
    )
    ax.plot(mean_internal_get_latest, label="MINIKUBE: internal")
    ax.fill_between(
        x_pos,
        mean_internal_get_latest - std_internal_get_latest,
        mean_internal_get_latest + std_internal_get_latest,
        alpha=0.15,
    )
    ax.plot(mean_external_get_latest, label="MINIKUBE: external")
    ax.fill_between(
        x_pos,
        mean_external_get_latest - std_external_get_latest,
        mean_external_get_latest + std_external_get_latest,
        alpha=0.15,
    )

    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f"{y / 1000:.0f}"))

    # Labeling
    ax.set_xlabel(x_label, fontproperties=medium_font_prop, fontsize=18)
    ax.set_ylabel("Duration (ms)", fontproperties=medium_font_prop, fontsize=18)
    ax.set_xticks(x_pos)
    ax.tick_params(axis="x", labelsize=15)
    ax.tick_params(axis="y", labelsize=15)
    ax.set_xticklabels(x)
    ax.legend(fontsize=16)
    ax.grid(True, axis="y", linestyle="--", alpha=0.6)

    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches="tight")
    plt.show()

## Latency - Single Kind, increasing resources

In [ ]:
(
    cases,
    mean_antiaffinity_get_latest,
    std_antiaffinity_get_latest,
    mean_antiaffinity_direct,
    std_antiaffinity_direct,
) = extract_results("../results/gke_antiaffinity/results_resource_latency.jsonl")
(
    _,
    mean_affinity_get_latest,
    std_affinity_get_latest,
    mean_affinity_direct,
    std_affinity_direct,
) = extract_results("../results/gke_affinity/results_resource_latency.jsonl")
(
    _,
    mean_internal_get_latest,
    std_internal_get_latest,
    mean_internal_direct,
    std_internal_direct,
) = extract_results("../results/minikube_internal/results_resource_latency.jsonl")
(
    _,
    mean_external_get_latest,
    std_external_get_latest,
    mean_external_direct,
    std_external_direct,
) = extract_results("../results/minikube_external/results_resource_latency.jsonl")
x = [case.resource_count for case in cases]

### Bar plot with error bars

In [ ]:
bar_plot(
    x,
    mean_antiaffinity_get_latest,
    std_antiaffinity_get_latest,
    mean_antiaffinity_direct,
    std_antiaffinity_direct,
    "Number of Secondaries",
    "../results/gke_antiaffinity/resource_latency_bar_plot.pdf",
)
bar_plot(
    x,
    mean_affinity_get_latest,
    std_affinity_get_latest,
    mean_affinity_direct,
    std_affinity_direct,
    "Number of Secondaries",
    "../results/gke_affinity/resource_latency_bar_plot.pdf",
    show=False,
)
bar_plot(
    x,
    mean_internal_get_latest,
    std_internal_get_latest,
    mean_internal_direct,
    std_internal_direct,
    "Number of Secondaries",
    "../results/minikube_internal/resource_latency_bar_plot.pdf",
    show=False,
)
bar_plot(
    x,
    mean_external_get_latest,
    std_external_get_latest,
    mean_external_direct,
    std_external_direct,
    "Number of Secondaries",
    "../results/minikube_external/resource_latency_bar_plot.pdf",
    show=False,
)

### Comparing different execution environments

In [ ]:
compare_plot(
    x,
    mean_antiaffinity_get_latest,
    std_antiaffinity_get_latest,
    mean_affinity_get_latest,
    std_affinity_get_latest,
    mean_internal_get_latest,
    std_internal_get_latest,
    mean_external_get_latest,
    std_external_get_latest,
    "Number of Resources",
    "../results/comparison_resource_latency_bar_plot.pdf",
)

## Latency - 100 resources, increasing number of kinds

In [ ]:
(
    cases,
    mean_antiaffinity_get_latest,
    std_antiaffinity_get_latest,
    mean_antiaffinity_direct,
    std_antiaffinity_direct,
) = extract_results("../results/gke_antiaffinity/results_kind_latency.jsonl")
(
    _,
    mean_affinity_get_latest,
    std_affinity_get_latest,
    mean_affinity_direct,
    std_affinity_direct,
) = extract_results("../results/gke_affinity/results_kind_latency.jsonl")
(
    _,
    mean_internal_get_latest,
    std_internal_get_latest,
    mean_internal_direct,
    std_internal_direct,
) = extract_results("../results/minikube_internal/results_kind_latency.jsonl")
(
    _,
    mean_external_get_latest,
    std_external_get_latest,
    mean_external_direct,
    std_external_direct,
) = extract_results("../results/minikube_external/results_kind_latency.jsonl")
x = [case.kind_count for case in cases]

### Bar plot with error bars

In [ ]:
bar_plot(
    x,
    mean_antiaffinity_get_latest,
    std_antiaffinity_get_latest,
    mean_antiaffinity_direct,
    std_antiaffinity_direct,
    "Number of Kinds",
    "../results/gke_antiaffinity/kind_latency_bar_plot.pdf",
)
bar_plot(
    x,
    mean_affinity_get_latest,
    std_affinity_get_latest,
    mean_affinity_direct,
    std_affinity_direct,
    "Number of Kinds",
    "../results/gke_affinity/kind_latency_bar_plot.pdf",
    show=False,
)
bar_plot(
    x,
    mean_internal_get_latest,
    std_internal_get_latest,
    mean_internal_direct,
    std_internal_direct,
    "Number of Kinds",
    "../results/minikube_internal/kind_latency_bar_plot.pdf",
    show=False,
)
bar_plot(
    x,
    mean_external_get_latest,
    std_external_get_latest,
    mean_external_direct,
    std_external_direct,
    "Number of Kinds",
    "../results/minikube_external/kind_latency_bar_plot.pdf",
    show=False,
)

In [ ]:
compare_plot(
    x,
    mean_antiaffinity_get_latest,
    std_antiaffinity_get_latest,
    mean_affinity_get_latest,
    std_affinity_get_latest,
    mean_internal_get_latest,
    std_internal_get_latest,
    mean_external_get_latest,
    std_external_get_latest,
    "Number of Kinds",
    "../results/comparison_kind_latency_bar_plot.pdf",
)

## Memory usage

Note: this will be done with synthetic data due to small size of Kubernetes resources

In [ ]:
y_get_latest = [0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 0]
y_direct = [0, 1, 21, 1, 21, 1, 21, 1, 21, 1, 21, 1, 0]
x = np.arange(len(y_direct))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.step(x, y_get_latest, label="get_latest", where="post")
ax.step(x, y_direct, label="direct", where="post")

# Labeling
ax.set_xlabel("Time steps", fontproperties=medium_font_prop, fontsize=18)
ax.set_ylabel(
    "Number of resources in memory", fontproperties=medium_font_prop, fontsize=18
)
ax.set_xticks(x)
ax.tick_params(axis="x", labelsize=13)
ax.tick_params(axis="y", labelsize=15)
ax.set_xticklabels(
    [
        "Start",
        "Get primary",
        "Enter scope 1",
        "Leave scope 1",
        "Enter scope 2",
        "Leave scope 2",
        "Enter scope 3",
        "Leave scope 3",
        "Enter scope 4",
        "Leave scope 4",
        "Enter scope 5",
        "Leave scope 5",
        "End",
    ]
)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
ax.legend(fontsize=16, loc="center right")
ax.grid(True, axis="y", linestyle="--", alpha=0.6)

plt.tight_layout()
plt.savefig("./memory_representation.pdf", dpi=300, bbox_inches="tight")
plt.show()